# 銀河のぼんやり白いもの：回帰（線形＋ポアソン）と不確実性（v2）

このノートは、**『銀河鉄道の夜』の世界観**を壊さずに、統計的学習とディープラーニングへの入口までを1本でつなぎます。

- 先生の説明：「星の集まりがぼうっと白く見える」＝レンズ模型で説明される“厚み”の効果
- アルビレオ観測所：「二つの球（サファイア／トパース）が重なって緑の両面凸レンズになる」＝観測の鮮明さ（overlap）

ここでのデータ：
- 1行 = 銀河鉄道の窓からの「観測記録（観測メモ付き）」
- `u` = 銀河帯のどこを見るか（中心に近いほど“厚い”）
- `overlap` = レンズの重なり（鮮明さ）
- `star_count` = 見える星の数（カウント）
- `haze_obs` = ぼんやり白さ（連続値）

> 深呼吸してから検証する：モデルの前に“観測条件”を疑う。これがこの教材の作法です。


> 作者用：oracle/event列を保持し、チェックセルを追加しています。

## 0) 文学的背景：賢治の科学観（短い補助線）

この教材が「こじつけ」に見えないために、導入で次の一点だけ共有すると効きます。

- 『銀河鉄道の夜』は幻想だけでなく、当時の天文学・物理学（相対性理論など）への関心が背景にある、と解説されることがある。  
  例：国立天文台の解説資料や、天文学者による読解・解説本の紹介（当時の科学史と賢治の年表比較など）。

（参考）
- 国立天文台（NAOJ）関連資料：『銀河鉄道の夜』に天文学・物理学が反映されている旨の解説（PDF資料）
- 日本天文学会の月報に掲載された書評：当時の天文学・物理学の状況と作品読解を結ぶ構成に触れている
- 「第四次延長（四次元）」の意識に関する研究論文（大学リポジトリPDF）

※このノートでは、作品の“科学っぽさ”を断定しません。  
ただ、**「観測から推定する」姿勢が物語と統計学で一致する**ことを、動機づけとして使います。


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.metrics import mean_squared_error, mean_poisson_deviance

# 追加：変数選択・多重共線性の簡易診断
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, RidgeCV


In [ ]:
# データ読み込み（v2：作者用は oracle と event を保持）
df = pd.read_csv("ginga_galaxy_haze_v2.csv")
df.tail(3)


In [ ]:
# 観測の分布（窓景色の“統計”）
fig, ax = plt.subplots()
ax.hist(df["star_count"], bins=40)
ax.set_title("star_count (見える星の数)")
plt.show()

fig, ax = plt.subplots()
ax.hist(df["haze_obs"], bins=40)
ax.set_title("haze_obs (ぼんやり白さ)")
plt.show()

fig, ax = plt.subplots()
ax.hist(df["overlap"], bins=30)
ax.set_title("overlap (レンズの重なり)")
plt.show()


In [ ]:
# 特徴量づくり：中心ほど厚い → u^2 を入れる
df2 = df.copy()
df2["u2"] = df2["u"]**2

base_cols = ["u","u2","overlap","depth_blue","f_white","f_blue","f_light","f_star","f_len"]
X_base = df2[base_cols]


In [ ]:
# 1) ぼんやり白さ（連続値）を線形回帰でモデル化
y = df2["haze_obs"]
Xtr, Xte, ytr, yte = train_test_split(X_base, y, test_size=0.25, random_state=0)

lin = LinearRegression().fit(Xtr, ytr)
pred = lin.predict(Xte)
print("LinearRegression MSE (haze):", mean_squared_error(yte, pred))

coef = pd.Series(lin.coef_, index=X_base.columns).sort_values(ascending=False)
coef


## 2) 特徴量エンジニアリング：青×白（depth_blue の活用）

物語では「青」と「白」のコントラストが重要です。  
仮説として、**青さ（depth_blue）が深いほど、“白いぼんやり”が打ち消される**（または別の形で変化する）と考えます。

そこで交互作用（interaction）項を入れてみます：
- `depth_blue * f_white`
- `depth_blue * overlap`
- `overlap * f_white`

さらに、変数が増えると **多重共線性**（似た情報の重複）も起きやすい。  
ISLR的には「解釈の難しさ・不安定さ」として扱うポイントです。


In [ ]:
# 交互作用項を追加
df2["blue_x_white"] = df2["depth_blue"] * df2["f_white"]
df2["blue_x_overlap"] = df2["depth_blue"] * df2["overlap"]
df2["overlap_x_white"] = df2["overlap"] * df2["f_white"]

cols = base_cols + ["blue_x_white","blue_x_overlap","overlap_x_white"]
X = df2[cols]
y = df2["haze_obs"]

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=0)

# まずは単純な線形回帰
lin2 = LinearRegression().fit(Xtr, ytr)
pred2 = lin2.predict(Xte)
print("LinearRegression MSE (haze, +interaction):", mean_squared_error(yte, pred2))

pd.Series(lin2.coef_, index=X.columns).sort_values(ascending=False)


In [ ]:
# 多重共線性の“気配”を簡易に見る：相関と条件数（condition number）
corr = pd.DataFrame(X).corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(7,6))
im = ax.imshow(corr.to_numpy(), vmin=-1, vmax=1)
ax.set_xticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90, fontsize=8)
ax.set_yticks(range(len(corr.columns)))
ax.set_yticklabels(corr.columns, fontsize=8)
ax.set_title("Correlation (features)")
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.show()

# 条件数（大きいほど、推定が不安定になりやすい）
Xs = StandardScaler().fit_transform(X)
cond = np.linalg.cond(Xs)
print("condition number:", cond)


In [ ]:
# 変数選択（Lasso）と安定化（Ridge）
# Lasso：不要な係数を0にしやすい（解釈の足場）
lasso = make_pipeline(StandardScaler(), LassoCV(cv=5, random_state=0, max_iter=10000))
lasso.fit(Xtr, ytr)
pred_l = lasso.predict(Xte)
print("Lasso MSE:", mean_squared_error(yte, pred_l))

# Ridge：係数を小さくして安定化（多重共線性に強い）
ridge = make_pipeline(StandardScaler(), RidgeCV(alphas=np.logspace(-3,3,30), cv=5))
ridge.fit(Xtr, ytr)
pred_r = ridge.predict(Xte)
print("Ridge MSE:", mean_squared_error(yte, pred_r))

# Lasso の係数（0になったものが“選ばれなかった”）
coef_l = pd.Series(lasso.named_steps["lassocv"].coef_, index=X.columns).sort_values(ascending=False)
coef_l


## 3) 「ぼんやり」＝観測ノイズ：overlap で分散が変わるか？

overlap が小さい（レンズが重ならない）ほど、滲みが増えて観測が不安定になる――という仮説を確認します。


In [ ]:
tmp = df2.copy()
tmp["over_bin"] = pd.qcut(tmp["overlap"], q=6, duplicates="drop")
g = tmp.groupby("over_bin")["haze_obs"].agg(["mean","var","count"]).reset_index()
g


## 4) 星の数（カウント）：線形回帰 vs ポアソン回帰（GLM）

星の数は **非負の整数**。  
「平均が上がると揺らぎも上がる（mean-variance）」という性質が自然に出やすいので、ポアソン回帰が似合います。


In [ ]:
df_cnt = df2.copy()
df_cnt["u2"] = df_cnt["u"]**2
Xc = df_cnt[["u","u2","overlap","depth_blue","f_white","f_blue","f_light","f_star","f_len",
             "blue_x_white","blue_x_overlap","overlap_x_white"]]
y = df_cnt["star_count"]

Xtr, Xte, ytr, yte = train_test_split(Xc, y, test_size=0.25, random_state=0)

lin = LinearRegression().fit(Xtr, ytr)
pred_lin = lin.predict(Xte)
print("Linear MSE (count):", mean_squared_error(yte, pred_lin))
print("neg prediction rate:", float((pred_lin < 0).mean()))

# ポアソン回帰は重い場合があるのでサブサンプルで
sub = 900
if len(Xtr) > sub:
    Xtr_p = Xtr.sample(sub, random_state=0)
    ytr_p = ytr.loc[Xtr_p.index]
else:
    Xtr_p, ytr_p = Xtr, ytr

pois = PoissonRegressor(alpha=1e-4, max_iter=1000).fit(Xtr_p, ytr_p)
pred_p = pois.predict(Xte)
print("Poisson deviance:", mean_poisson_deviance(yte, pred_p))

pd.Series(pois.coef_, index=Xc.columns).sort_values(ascending=False)


In [ ]:
# mean-variance の手触り：近い空どうし（uのビン）でまとめる
tmp = df2.copy()
tmp["u_bin"] = pd.qcut(tmp["u"], q=10, duplicates="drop")
mv = tmp.groupby("u_bin")["star_count"].agg(["mean","var","count"]).reset_index()
mv


## 5) 外れ値（Outlier）を“物語”として読む：カムパネルラ消失

v2データには、意図的に **1点だけ強い外れ値**を混ぜています（event列で確認できます）。  
利用者には伏せ、残差から見つけさせます。


In [ ]:
# 作者用：event列でチェック
mu = pois.predict(Xc)
res = (df_cnt["star_count"].to_numpy() - mu) / np.sqrt(np.clip(mu, 1e-6, None))
idx = np.argsort(np.abs(res))[::-1][:15]
df_cnt.loc[idx, ["obs_id","event_flag","event_name","star_count","u","overlap","memo"]]


## 6) D2Lへの橋渡し：もし“星の配置”が画像だったら？

ここまでは tabular（表形式）でした。  
もし各観測が **星図画像**（たとえば 32×32 の星の散布）として与えられたら、次の流れになります：

- 入力：画像（星の点の配置）
- 出力：star_count（カウント）や haze_obs（連続）
- モデル：CNN
- 損失：カウントなら Poisson NLL など

下のセルは「まずはPyTorchで tabular を学習」する最小例です。  
（環境にtorchが無い場合はスキップします）


In [ ]:
try:
    import torch
    import torch.nn as nn
    from torch.utils.data import TensorDataset, DataLoader

    # データ（サブサンプル）
    d = df2.sample(min(1000, len(df2)), random_state=0).copy()
    feats = ["u","u2","overlap","depth_blue","f_white","f_blue","f_light","f_star","f_len"]
    X_t = torch.tensor(d[feats].to_numpy(), dtype=torch.float32)
    y_t = torch.tensor(d["star_count"].to_numpy(), dtype=torch.float32).view(-1,1)

    ds = TensorDataset(X_t, y_t)
    dl = DataLoader(ds, batch_size=64, shuffle=True)

    # Poisson 回帰（log-link）：線形→exp で λ を作る
    model = nn.Linear(X_t.shape[1], 1)

    # Poisson NLL（log_input=Falseでλ入力）
    loss_fn = nn.PoissonNLLLoss(log_input=False, full=True)

    opt = torch.optim.Adam(model.parameters(), lr=1e-2)

    for epoch in range(8):
        total = 0.0
        for xb, yb in dl:
            lam = torch.exp(model(xb)) + 1e-6
            loss = loss_fn(lam, yb)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total += loss.item() * len(xb)
        print("epoch", epoch, "loss", total/len(ds))

    # 係数を見る（どの入力がλを何倍にするか）
    w = model.weight.detach().cpu().numpy().ravel()
    b = float(model.bias.detach().cpu().numpy().ravel()[0])
    print("bias", b)
    print(dict(zip(feats, w)))

except Exception as e:
    print("torch not available or failed:", e)


## メモやること（短メモ＋コード）

1. **星の数（star_count）**  
   - 線形回帰とポアソン回帰のどちらが自然か。  
   - 対数リンクの係数を「何倍になるか」の言葉で説明し、物語に接続する。

2. **ぼんやり白さ（haze_obs）**  
   - overlap が小さいほど分散が大きい（ぼんやり）ことを示す。  
   - interaction項（青×白 など）を入れたとき、解釈はどう変わるか。

3. **外れ値（観測の断絶）**  
   - 残差で外れ値候補を1つ挙げ、観測メモ（memo）と照らし、どういう“断絶”が起きたのかを説明する。

4. **D2Lへの問い**  
   - もし星の配置が画像なら、どんなCNNと損失を使うか（1段落で設計案）。


## （作者用）oracleでのチェック（最小）

- `thickness_true` があるので、u→thickness の形を描く
- `event` で外れ値が意図的に混ざっていることを確認する


In [ ]:
# u -> thickness_true（先生の凸レンズ模型の形）
fig, ax = plt.subplots()
ax.scatter(df["u"], df["thickness_true"], s=5)
ax.set_title("u -> thickness_true（先生の凸レンズ模型）")
ax.set_xlabel("u")
ax.set_ylabel("thickness_true")
plt.show()

# event の確認
df[df["event_flag"]==1][["obs_id","event_name","star_count","haze_obs","u","overlap","memo"]]
